# Representações Distribuídas de Texto e Modelagem de Tópicos

<b> DOCENTES: </b>
<pre>
     Prof. Renato Rocha de Souza
     Prof. Mauricio Barcellos Almeida
     Profa. Renata Maria Abrantes Baracho Porto
</pre>

<b> DISCENTE: </b> Jeanne Louize Emygdio </br>

<b> OBJETIVO: </b> Identificar artigos científicos disponíveis na base Medline voltados à temática de uso do padrão OpenEHR - Uma ontologia para modelagem de dados médicos e que possam compor meu acervo particular de referencial teórico para a escrita da tese. 

<b> Estratégia selecionada: </b>  Modelagem de tópicos e clusterização de documentos. 

<b> Etapas: </b>
<pre>
    1.	Finalização do curso de Python disponibilizado pelo Prof. Renato
    2.	Instalação e configuração da plataforma de desenvolvimento
    3.	Fork da área de MMD do Prof. Renato no Github
    4.	Estudo do notebook 4.2_TopicModeling_Clustering_Documents.ipynb
    5.	Importação das bibliotecas necessárias
    6.	Desenvolvimento do novo notebook de acordo com as passagens previstas no 4.2.
</pre>

<b>Resultados esperados: </b>
<pre>
	- Desenvolvimento de habilidades de programação na linguagem Python;
	- Melhor compreensão das estratégias de obtenção de documentos, análises textuais e clusterização;
	- Montagem de biblioteca de artigos relevantes na temática selecionada de forma ágil.
 </pre>

In [7]:
# urllib - Módulo que reúne funções de alto nível para acesso à internet. 
# Documentation: https://docs.python.org/3/library/urllib.html | 
#                https://docs.python.org/3/library/urllib.request.html#module-urllib.request
import urllib as url                 

# BeautifulSoup - Biblioteca utilizada para extrair dados de arquivos HTML e XML.
# Documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup as bs

# NLTK - Biblioteca para trabalhar com processamentos de linguagem natural
# Documentation: https://pypi.python.org/pypi/nltk
import nltk
nltk.download('punkt')
from nltk.corpus       import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize     import word_tokenize

# Biblioteca para auxílio à contagem de freqência de ocorrência de palavras em um texto
from collections import Counter

# Bilioteca para lidar com processamento de strings através de expressões regulares
import re 

# Biblioteca que disponibiliza recursos para modelagem de tópicos
import gensim

#from selenium import webdriver

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Louize\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


C:\Users\Louize\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
# Acessando a página do Portal da Capes. O objetivo é pesquisar a base PUBMED.
# Realizei acesso com e sem autenticação e o resultado da pesquisa foi o mesmo. A codificação segue sem autenticação.
# O link alvo é: https://www.ncbi.nlm.nih.gov/pubmed/
# Termo pesquisado: OpenEHR. Encontrei 148 artigos.
# Outros termos para pesquisa futura: (((ontology[Title]) OR ontology[Abstract]) AND (owl OR 'web ontology' OR 'web ontologies') 

# Lógica utilizada descrita por linhas: Recuperar conteúdo da página usando urllib, e visualizar resultados tratados 
#                                       pela biblioteca BeautifulSoup.
# L20: Estrutura de controle para tratar adequadamente possíveis erros na execução dos comandos (try...except)
# L21: Variável v_terbus recebe a string que vou usar para buscar o termo OpenEHR no título e abstract
# L22: Acesso à url completa da página acrescida do conteúdo da variável v_terbus, utilizando a função urlib
# L23: Execução da biblioteca BeautifulSoup sobre a Url acessada para estudar os elementos da página e extrair os 
#      desejados. O resultado é armazenado na variável v_soup. O comando print(v_soup.prettify()) permite ver toda 
#      a estrutura da página armazenado na variável.
# L24: Declaração do vetor v_titles para armazenamento dos títulos encontrados
# L27 a 29: Após conhecimento dos elementos, busquei as classes que tivessem o valor title e que estivessem dentro de uma 
#      tag <p> (parágrafo) e, em seus resultados, busquei os conteúdos das tags <a>, que me levariam aos títulos dos 
#      artigos. Estruturei dois loops para analisar os dois resultados aninhados. E em seguida armazenei cada título no 
#      vetor v_titles.

try:   
   v_terbus = '(OpenEHR[Title]+OR+OpenEHR[Abstract])' 
   v_url    = url.request.urlopen('https://www.ncbi.nlm.nih.gov/pubmed/?term='+v_terbus) 
   v_soup   = bs(v_url,'html.parser')
   v_titles = [] # Declaração de um vetor para armazenar os títulos encontrados
                                                    
   for v_elemento in v_soup.find_all('p', class_='title'):                 
      for n in bs(v_elemento.find('a').text): # Tentando extrair apenas a parte textual de cada trecho html
          v_titles.append(n)        
except url.error:
   print('Falha na conexão!')
   exit(1) #interrompe  a execução do programa              

C:\Users\Louize\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Louize\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [13]:
# Conferindo quantos títulos de artigos foram armazenados no vetor v_title
# Problema encontrado: não consegui ainda fazer o sistema localizar os 148 artigos, ele só está considerando os que 
# foram exibidos na primeira página da busca (20 artigos).
print("Number of documents:",len(v_titles))

Number of documents: 20


In [16]:
# Visualizando os artigos armazenados
# Comentário: Não consegui ainda remover as tags com BeautifulSoup, então utilizei a biblioteca 're' e funcionou!
#             Este código deverá ser otimizado no futuro
v_count = 0
while v_count <= 19:
   v_titles[v_count] = (re.sub('<[^>]+?>', '', v_titles[v_count]))
   print(v_titles[v_count])
   v_count += 1

Special Topic Interoperability and EHR: Combining openEHR, SNOMED, IHE, and Continua as approaches to interoperability on national eHealth.
Bridging the Gap Between the Social and the Technical: The Enrolment of Socio-Technical Information Architects to Cope with the Two-Level Model of EPR Systems.
Domain Modeling and Application Development of an Archetype- and XML-based EHRS. Practical Experiences and Lessons Learnt.
Applying an Archetype-Based Approach to Electroencephalography/Event-Related Potential Experiments in the EEGBase Resource.
Querying Archetype-Based Electronic Health Records Using Hadoop and Dewey Encoding of openEHR Models.
Introducing a Method for Transformation of Paper-Based Research Data into Concept-Based Representation with openEHR.
Integration of Hospital Information and Clinical Decision Support Systems to Enable the Reuse of Electronic Health Record Data.
Structuring Legacy Pathology Reports by openEHR Archetypes to Enable Semantic Querying.
A Survey of Standa

In [17]:
# Utilizando a biblioteca nltk para obter as palavras dos títulos separadas em tokens
v_tokens = [[w.lower() for w in nltk.word_tokenize(text)] for text in v_titles]
print(v_tokens)

[['special', 'topic', 'interoperability', 'and', 'ehr', ':', 'combining', 'openehr', ',', 'snomed', ',', 'ihe', ',', 'and', 'continua', 'as', 'approaches', 'to', 'interoperability', 'on', 'national', 'ehealth', '.'], ['bridging', 'the', 'gap', 'between', 'the', 'social', 'and', 'the', 'technical', ':', 'the', 'enrolment', 'of', 'socio-technical', 'information', 'architects', 'to', 'cope', 'with', 'the', 'two-level', 'model', 'of', 'epr', 'systems', '.'], ['domain', 'modeling', 'and', 'application', 'development', 'of', 'an', 'archetype-', 'and', 'xml-based', 'ehrs', '.', 'practical', 'experiences', 'and', 'lessons', 'learnt', '.'], ['applying', 'an', 'archetype-based', 'approach', 'to', 'electroencephalography/event-related', 'potential', 'experiments', 'in', 'the', 'eegbase', 'resource', '.'], ['querying', 'archetype-based', 'electronic', 'health', 'records', 'using', 'hadoop', 'and', 'dewey', 'encoding', 'of', 'openehr', 'models', '.'], ['introducing', 'a', 'method', 'for', 'transfor

In [20]:
# Criação de um dicionário da lista dos documentos encontrados, baseado em seus títulos
dictionary = gensim.corpora.Dictionary(v_tokens)

for i in range(len(dictionary)):
    print(i, dictionary[i])

print("Number of words in dictionary:",len(dictionary))

0 special
1 topic
2 interoperability
3 and
4 ehr
5 :
6 combining
7 openehr
8 ,
9 snomed
10 ihe
11 continua
12 as
13 approaches
14 to
15 on
16 national
17 ehealth
18 .
19 bridging
20 the
21 gap
22 between
23 social
24 technical
25 enrolment
26 of
27 socio-technical
28 information
29 architects
30 cope
31 with
32 two-level
33 model
34 epr
35 systems
36 domain
37 modeling
38 application
39 development
40 an
41 archetype-
42 xml-based
43 ehrs
44 practical
45 experiences
46 lessons
47 learnt
48 applying
49 archetype-based
50 approach
51 electroencephalography/event-related
52 potential
53 experiments
54 in
55 eegbase
56 resource
57 querying
58 electronic
59 health
60 records
61 using
62 hadoop
63 dewey
64 encoding
65 models
66 introducing
67 a
68 method
69 for
70 transformation
71 paper-based
72 research
73 data
74 into
75 concept-based
76 representation
77 integration
78 hospital
79 clinical
80 decision
81 support
82 enable
83 reuse
84 record
85 structuring
86 legacy
87 pathology
88 report

In [ ]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
for d in corpus:
    print(d)

In [86]:
# Descobrindo a frequência dos termos encontrados
# 1a tentativa: função FreqDist() da biblioteca NLTK FreqDist(v_titles)    ---> Não funcionou!
# 2a tentativa: função Counter da biblioteca collections: as frequências da palavras foram analisadas apenas dentro 
# de cada título, como pode ser visto abaixo. 
v_count = 0

while v_count <= 19:
   contador = Counter(v_tokens[v_count]) 
   for i in contador.items():
     print(i) 
   v_count += 1

('special', 1)
('topic', 1)
('interoperability', 2)
('and', 2)
('ehr', 1)
(':', 1)
('combining', 1)
('openehr', 1)
(',', 3)
('snomed', 1)
('ihe', 1)
('continua', 1)
('as', 1)
('approaches', 1)
('to', 1)
('on', 1)
('national', 1)
('ehealth', 1)
('.', 1)
('bridging', 1)
('the', 5)
('gap', 1)
('between', 1)
('social', 1)
('and', 1)
('technical', 1)
(':', 1)
('enrolment', 1)
('of', 2)
('socio-technical', 1)
('information', 1)
('architects', 1)
('to', 1)
('cope', 1)
('with', 1)
('two-level', 1)
('model', 1)
('epr', 1)
('systems', 1)
('.', 1)
('domain', 1)
('modeling', 1)
('and', 3)
('application', 1)
('development', 1)
('of', 1)
('an', 1)
('archetype-', 1)
('xml-based', 1)
('ehrs', 1)
('.', 2)
('practical', 1)
('experiences', 1)
('lessons', 1)
('learnt', 1)
('applying', 1)
('an', 1)
('archetype-based', 1)
('approach', 1)
('to', 1)
('electroencephalography/event-related', 1)
('potential', 1)
('experiments', 1)
('in', 1)
('the', 1)
('eegbase', 1)
('resource', 1)
('.', 1)
('querying', 1)
('arc

In [101]:
# Descobrindo a frequência dos termos encontrados
# Passos: 
# 1: Transformar os títulos que estão organizados de forma distinta em v_titles, em um único texto longo para que se 
#    identificar a frequência dos termos entre os títulos dos artigos encontrados já que não estou trabalhando com 
#    os conteúdos dos documentos.
v_count = 0
v_texto = ''

while v_count <= 19:
   v_texto = v_texto + '"'+ v_titles[v_count] +'",' 
   v_count += 1

print(v_texto)


#   if v_count == 0: 
#      v_texto = v_texto + '"'+ v_titles[v_count] +'",'
#   elif v_count >= 1:
#      v_texto = v_texto + ' ' + v_titles[v_count]


"Special Topic Interoperability and EHR: Combining openEHR, SNOMED, IHE, and Continua as approaches to interoperability on national eHealth.","Bridging the Gap Between the Social and the Technical: The Enrolment of Socio-Technical Information Architects to Cope with the Two-Level Model of EPR Systems.","Domain Modeling and Application Development of an Archetype- and XML-based EHRS. Practical Experiences and Lessons Learnt.","Applying an Archetype-Based Approach to Electroencephalography/Event-Related Potential Experiments in the EEGBase Resource.","Querying Archetype-Based Electronic Health Records Using Hadoop and Dewey Encoding of openEHR Models.","Introducing a Method for Transformation of Paper-Based Research Data into Concept-Based Representation with openEHR.","Integration of Hospital Information and Clinical Decision Support Systems to Enable the Reuse of Electronic Health Record Data.","Structuring Legacy Pathology Reports by openEHR Archetypes to Enable Semantic Querying.","A

In [102]:
# Descobrindo a frequência dos termos encontrados
# Passos: 
# 2: Executar a contagem da frequência com a função counter()
# 3: Exibir as frequências de cada termo

v_frequencia = Counter(v_texto) 
for i in v_frequencia.items():
  print(i)
  v_count += 1

('"', 42)
('S', 18)
('p', 87)
('e', 234)
('c', 83)
('i', 132)
('a', 165)
('l', 83)
(' ', 295)
('T', 7)
('o', 163)
('I', 13)
('n', 164)
('t', 155)
('r', 110)
('b', 13)
('y', 36)
('d', 58)
('E', 37)
('H', 26)
('R', 30)
(':', 8)
('C', 10)
('m', 36)
('g', 42)
(',', 25)
('N', 2)
('O', 4)
('M', 13)
('D', 20)
('u', 35)
('s', 93)
('h', 51)
('.', 21)
('B', 11)
('G', 6)
('w', 9)
('f', 32)
('-', 16)
('A', 19)
('L', 9)
('v', 12)
('P', 6)
('X', 1)
('x', 3)
('/', 1)
('Q', 2)
('U', 2)
("'", 1)
('k', 1)
('2', 3)
('K', 1)
('W', 1)
('F', 1)
('&', 1)
('J', 1)
('0', 1)
('1', 1)
('6', 1)
('V', 1)
('8', 1)
('9', 1)
('4', 2)
('3', 1)
('5', 1)
('7', 1)
